# SetUp for YOLACT++
YOLACT: <br>
https://github.com/dbolya/yolact

Reference: <br>
https://qiita.com/PoodleMaster/items/d0bf9cac77e5af61c972
<br>
https://note.com/thedesignium/n/n0ba51afe9776

In [ ]:
# You need to set the runtime type to GPU.

# Check GPU Info
print("=====GPU=====")
!nvidia-smi

print("=====GPU MEMORY=====")
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
    process = psutil.Process(os.getpid())
    print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
    print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

# You can set options in Colab Pro to access high memory VMs as soon as they are available.
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('\nYour runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

In [ ]:
# Install libraries for YOLACT
!pip install cython
!pip install opencv-python pillow pycocotools matplotlib
!pip install torchvision==0.5.0
!pip install torch==1.4.0

In [ ]:
# Download YOLACT
%cd /content
!git clone https://github.com/dbolya/yolact.git

In [ ]:
# Compile deformable convolutional layers (from DCNv2) for YOLACT++
%cd /content/yolact/external/DCNv2
!python setup.py build develop

In [ ]:
# Download files from Google Drive using Python 2 or Python 3
# https://github.com/chentinghao/download_google_drive

%cd /content
!git clone https://github.com/chentinghao/download_google_drive.git

!mkdir -p '/content/yolact/weights'
%cd download_google_drive

# Download weights of YOLACT++ models(Resnet50-FPN)
weights = '/content/yolact/weights/yolact_plus_resnet50_54_800000.pth'
!python download_gdrive.py 1ZPu1YR2UzGHQD0o1rEqy-j5bmEm3lbyP '$weights'

# # Download weights of YOLACT++ models(Resnet101-FPN)
# weights = '/content/yolact/weights/yolact_plus_base_54_800000.pth'
# !python download_gdrive.py 15id0Qq5eqRbkD-N3ZjDZXdCvRyIaHpFB '$weights'

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

%cd /content/yolact

# Image

In [ ]:
# Set path to the image file.
input_image  = "/content/drive/MyDrive/hoge/input.jpg"
output_image = "/content/drive/MyDrive/hoge/output.jpg"

# Process an image and save it to another file.
!python eval.py --trained_model='$weights'\
                --config=yolact_plus_resnet50_config\
                --score_threshold=0.15\
                --top_k=15\
                --image="$input_image":"$output_image"

In [ ]:
# Set path to the directory containing the image files.
input_dir  = "/content/drive/MyDrive/hoge/Input"
output_dir = "/content/drive/MyDrive/hoge/Output"

# Process a whole folder of images.
!python eval.py --trained_model='$weights'\
                --config=yolact_plus_resnet50_config\
                --score_threshold=0.15\
                --top_k=15\
                --images='$input_dir':'$output_dir'

# Video

In [ ]:
# # Set path to the video file.
input_video  = "/content/drive/MyDrive/hoge/input.mp4"
output_video = "/content/drive/MyDrive/hoge/output.mp4"

# Process a video and save it to another file.
!python eval.py --trained_model='$weights'\
                --score_threshold=0.15\
                --top_k=15\
                --video_multiframe=1\
                --video='$input_video':'$output_video'